In [1]:
import json
from tqdm import tqdm
from glob import glob

In [2]:
files = glob('mixtral-rag-question-factually-wrong/*.json')
files.extend(glob('mixtral-rag-question-factually-wrong-textbook/*.json'))
len(files)

146548

In [3]:
import re

mapping = {}
for f in glob('questions.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue

            n = l['r']['result']
            hypens = re.findall('\w+ -\w+', n)
            for h in hypens:
                splitted = h.split('-')
                if len(splitted) != 2:
                    continue
                splitted = [s.strip() for s in splitted]
                splitted = '-'.join(splitted)
                n = n.replace(h, splitted)
            mapping[l['src']] = n
            
len(mapping)

69009

In [4]:
done = set()
prepared = []

for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    if data[0] not in done:
        output = data[-1]
        splitted = output.strip().split('\n')
        splitted = [s for s in splitted if '.' if s and '?' in s]
        splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
        splitted = [s for s in splitted if len(s) > 3]
        splitted = [s[1:] if s[0] == '"' else s for s in splitted]
        q = [s[:-1] if s[-1] == '"' else s for s in splitted]
        q = [s for s in q if len(s) > 10]
        q_join = '\n'.join([f'{no + 1}. {q}' for no, q in enumerate(q)])
        done.add(data[0])
        prepared.append({
            'context': data[0],
            'questions': q,
            'questions_join': q_join,
            'questions_join_ms': mapping.get(q_join),
        })

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 146548/146548 [00:04<00:00, 32605.60it/s]


In [5]:
len(prepared)

70647

In [7]:
with open('gather-questions.jsonl', 'w') as fopen:
    for d in prepared:
        fopen.write(f'{json.dumps(d)}\n')

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='gather-questions.jsonl',
    path_in_repo='gather-questions.jsonl',
    repo_id='mesolitica/mixtral-factual-QA',
    repo_type='dataset',
)

gather-questions.jsonl:   0%|          | 0.00/819M [00:00<?, ?B/s]